# ЦЕЛЬ
### Подготовить основу рекомендательной системы.

## ЗАДАЧИ:

1. Определить по какому сценарию будет осуществляться рекомендация курсов.
2. Подготовить данные по проданным курсам в разрезе пользователей (с помощью SQL).
3. Анализ данных. 
4. Определение рекомендаций и критериев.
5. Составление итоговой таблицы с рекомендациями.

### 1. Определим по какому сценарию будет осуществляться рекомендация курсов.

Найдем все варианты приобретенных пар курсов. Оставим для рекомендации наиболее популярные для каждого курса и преодоливающие определенный порог по частоте продаж.

Осталось решить, что рекомендовать в остальных случаях: когда курс не пользуется большим спросом у покупателей купивших 2+ курса или курс перестал быть актуальным.<br>
У неактуальных не будет продаж за последние месяцы - полгода, а у непопулярных курсов не будет большой истории по парам продаж (низкая частота их покупок).

Варианты:
1. Самые популярные за весь наблюдаемый период. <br>
-- Среди них могут быть те, что уже не актуальны = нулевые продажи. <br>
2. Самые новые.<br>
-- Курс может быть узкоспециализирован и не подойдёт абсолютному большинству = низкие продажи.
3. Самые популярные за последние полгода рассматриваемого периода.<br>
++ Получим только актуальные и массовые курсы, и у новых курсов есть шанс пробиться в топ выборки.

Остановимся на третьем варианте.

Следовательно, кроме курсов в разрезе пользователей, купивших более одного курса, добавляем еще признак месяца покупки курса в запросе.

### 2. Подготовка данных по проданным курсам в разрезе пользователей

In [1]:
# установим необходимые библиотеки
import psycopg2.extras
import pandas as pd
import numpy as np
import itertools 
from collections import Counter


In [2]:
# выполним SQL запрос на выемку интересующих данных
def getBuyersMultipleCourse():
    query = '''
WITH  dataset AS
      /* формируем CTE, содержащую пользователей оплативших курсы */
      (SELECT DISTINCT user_id, 
              resource_id as course_id,  -- сразу переименуем столбец для удобства 
              MIN(DATE_TRUNC('month', purchased_at)::date) AS purchase_date -- используем MIN, чтобы избавится от дублей покупок пользователя
         FROM final.carts 
        INNER JOIN final.cart_items ON carts.id = cart_items.cart_id
        WHERE state = 'successful'  -- только покупки
          AND resource_type = 'Course'  -- только курсы
        GROUP BY 1, 2)

SELECT user_id, 
       course_id,  
       purchase_date
  FROM dataset
 WHERE user_id IN  -- фильтр по списку "квалифицированных" покупателей
        (/* из CTE получаем список пользователей с 2-мя и более покупками курсов */
        SELECT user_id
          FROM dataset
         GROUP BY user_id
        HAVING count(distinct course_id) > 1)
 ORDER BY 1, 2
    '''.format()
    conn = psycopg2.connect(
        "dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data


# выполняем функцию запроса и записываем полученные данные в датафрейм
df = pd.DataFrame(getBuyersMultipleCourse())

# Внимание: Если в ходе выполнения возникло сообщение об ошибке,
# то повторите выполнение ячейки через несколько секунд: бывают неполадки с подключением к серверу.


### 3. Анализ данных

In [3]:
df.head()  # выведем первые 5 строк


,user_id,course_id,purchase_date
0,51,516,2017-01-01
1,51,1099,2018-06-01
2,6117,356,2017-06-01
3,6117,357,2017-06-01
4,6117,1125,2018-08-01


In [4]:
# Смотрим информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   user_id        34074 non-null  int64 
 1   course_id      34074 non-null  int64 
 2   purchase_date  34074 non-null  object
dtypes: int64(2), object(1)
memory usage: 798.7+ KB


В датасете всего 34 074 строк. <br>
Пропущенные значения отсутствуют. <br>
DISTINCT в базовом CTE гарантирует отсутствие дубликатов в итоговом запросе.

Столбец 'purchase_date' имеет типа данных object, необходимо привести его к типу данных datetime для дальнейшей корректной работы с ним.

In [5]:
df['purchase_date'] = pd.to_datetime(df['purchase_date'], format='%Y-%m-%d')


### 4. Определение рекомендаций и критериев.

Найдем топ-2 курсов по продажам за последние полгода рассматриваемого периода.

In [6]:
# Создадим отдельный датафрейм (исходный еще пригодится) 
# и перенесём в него записи продаж за последние полгода
df2 = df[df['purchase_date'] >= '2018-07-01']

# подсчитываем кол-во продаж по курсам
top_course = df2.groupby('course_id')['purchase_date'].count().reset_index()
# переименуем столбец
top_course.rename(columns={'purchase_date': 'count'}, inplace=True)
# отсортируем по убыванию
top_course = top_course.sort_values(
    by='count', ascending=False).reset_index(drop=True)
top_course.head()


,course_id,count
0,566,551
1,840,380
2,1103,359
3,1125,353
4,794,330


Определены id курсов для рекомендаций, в случае отсутствия таковых используем 566 и 840.

Создаём рекомендации курсов на основе предыдущих покупок.

Для этого необходимо определить частоты всех пар курсов приобретенных среди пользователей.

Сперва сгруппируем курсы по пользователям в новый датафрейм.

In [7]:
# С помощью метода np.unique получим уникальные и отсортированные по возрастанию значения курсов.
group_df = df.groupby('user_id')['course_id'].apply(lambda x:list(np.unique(x))).reset_index()
group_df

,user_id,course_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 864, 1129]"
12653,2187601,"[356, 553, 571, 765, 912]"
12654,2188926,"[515, 743]"


Найдём все возможные комбинации пар курсов приобретенных пользователями.

In [8]:
# используем цикл с функцией Combinations из библиотеки Itertools.
list_courses = list()
for course in group_df['course_id']:
    # проходим по перечню курсов каждого клиента, формируя из них все возможные комбинации пар
    for x in itertools.combinations(course, 2):  
        list_courses.append(x)  # добавляем пары в список

list_courses[0:5]


[(516, 1099), (356, 357), (356, 1125), (357, 1125), (553, 1147)]

С помощью счетчика Counter из библиотеки collections создаём словарь, в котором ключи - пары курсов, значения - их частота. 

In [9]:
uniq_pairs_courses = Counter(list_courses)  # создаёт словарь: ключи - пары курсов, значения - их количество
print("Количество уникальных пар курсов в покупках клиентов:", len(uniq_pairs_courses))

# Сортируем словарь по уменьшению количества покупок
sorted_pairs = dict(uniq_pairs_courses.most_common())
# Посмотрим наиболее популярные пары курсов
top_pairs = list(sorted_pairs.items())[:5]
print(f'Топ-5 пар курсов: \n {top_pairs}')

Количество уникальных пар курсов в покупках клиентов: 3989
Топ-5 пар курсов: 
 [((551, 566), 797), ((515, 551), 417), ((489, 551), 311), ((523, 551), 304), ((566, 794), 290)]


В каждой из четырех наиболее частотных пар курсов есть id 551. <br>
Однако, данного id нет среди топ-5 курсов за последние полгода.<br>
Присмотримся к нему.


In [10]:
c551 = top_course[top_course.course_id == 551]
c551

,course_id,count
117,551,2


Он на 117 месте по продажам курсов за последние полгода, всего с двумя покупками. 

Самый популярный курс за всё время оказался неактуальным в последние полгода минимум, либо был переработан и сменил id. <br> 
(следует уточнить у владеющего информацией сотрудника, но за неимением такой возможности ...)

В любом из этих вариантов советовать данный курс уже не имеет смысла. Поэтому убираем его из дальнейших рекомендаций, меняя значения с курсом 551 на 0, чтобы подбор осуществлялся среди актуальных курсов.
А при отсутствии таковых, будем использовать наиболее популярные среди актуальных (id 566 или 840).


In [11]:
for i in sorted_pairs:  # проходим по элементам словаря
    for k in sorted_pairs.keys():  # проверяем в ключе наличие курса 551
        if k[0] == 551 or k[1] == 551:  # если имеется
            sorted_pairs.update({k:0})  # обнуляем значение пары
        
# проверим результат 
list(sorted_pairs.items())[:5]

[((551, 566), 0),
 ((515, 551), 0),
 ((489, 551), 0),
 ((523, 551), 0),
 ((566, 794), 290)]

### Выбор минимальной границы для частотности рекомендаций. 

Определим условную границу встречаемости пар как достаточную, чтобы считать сложившиеся пары взаимосвязанными (покупка в такой паре не была случайной). 
Для этого используем процентиль 75.

In [12]:
percentile_score = np.percentile(list(sorted_pairs.values()), 75)  
print(f"75 процентиль соответствует частоте покупок в: {percentile_score}")


75 процентиль соответствует частоте покупок в: 8.0


Частоту пар курсов ниже 8 не будем учитывать. 


### 5. Составление итоговой таблицы с рекомендациями.

У нас есть словарь из сложившихся пар курсов и их частоты, а так же границе, ниже которой их не учитываем как рекомендацию.<br>
Применим эту информацию для формирования первичных рекомендаций.

In [13]:
def recommend(course):
    list = []  # создаём пустой список
    for i in sorted_pairs.keys():  # проходим по ключам словаря
        if i[0] == course and sorted_pairs[i] > 7:  # если id курса идёт первым в паре курсов
            # добавляем в созданный список саму пару и её значение
            list.append((i[0], i[1], sorted_pairs[i]))
        if i[1] == course and sorted_pairs[i] > 7:  # если id курса идёт вторым в паре курсов
            # добавляем в созданный список саму пару и её значение
            list.append((i[1], i[0], sorted_pairs[i]))
    # сортируем найденные пары по значению в убывающем порядке
    sorted_course_list = sorted(list, key=lambda x: x[2], reverse = True)
    return sorted_course_list[:2]  # оставляем топ-2 пары, если такие были получены


In [14]:
# пример выполнении функции по подбору наиболее частых пар для определенного курса
recommend(757)

[(757, 566, 48), (757, 490, 34)]

Посмотрим сколько пропусков значений на данном этапе:

In [15]:
# Создадим список из id курсов и отсортируем его для дальнейшего перебора значений
list_course_id = df['course_id'].sort_values().unique()

# Посчитаем у скольких курсов отсутсвуют обе рекомендации (частота пар была менее 5-ти раз):
missing_1 = 0
missing_2 = 0
for i in list_course_id:
    if len(recommend(i)) == 0:
        missing_2 += 1
    elif len(recommend(i)) == 1:
        missing_1 += 1
print(f"Количество курсов без одной рекомендации: {missing_1}")
print(f"Количество курсов без двух рекомендаций: {missing_2}")


Количество курсов без одной рекомендации: 5
Количество курсов без двух рекомендаций: 21


Пора заполнить эти пробелы, добавиив рекомендации на наиболее популярные за последние полгода наблюдаемого периода курсы (id 566 и 840).

Если уполномоченное лицо решит изменить выбор курсов на другой, то для этого достаточно поменять значение переменных rec1 и rec2 в коде ниже.<br>

In [16]:
# создаём датафрейм под таблицу с рекомендациями
recommend_df = pd.DataFrame(columns = ['Recommendation 1', 'Recommendation 2']) 
# Присваиваем переменным значения топ-2 курсов по продажам за полгода. 
rec1 = 566
rec2 = 840

for i in list_course_id:
    if len(recommend(i)) == 2:  # применяем функцию и если выдаёт 2 рекомендации для курса
        recommend_df.loc[i] = [recommend(i)[0][1], recommend(i)[1][1]]  
    elif len(recommend(i)) == 1 and recommend(i) == rec1:
        # в случае только с одной рекомендацией из частых пар равной 566 (rec1)
        # добавляем второй по популярности
        recommend_df.loc[i] = [recommend(i)[0][1], rec2]
    elif len(recommend(i)) == 1:
        # применяем функцию и если выдаёт 1 рекомендацию для курса
        # записываем одну рекомендацию из "частых" пар + самый популярный курс
        recommend_df.loc[i] = [recommend(i)[0][1], rec1]
    else:
        # остальным - рекомендовать два самый популярных за полгода
        recommend_df.loc[i] = [rec1, rec2]

recommend_df.columns.names = ['Basic_course']
display(recommend_df)


Basic_course,Recommendation 1,Recommendation 2
356,571,357
357,571,356
358,570,752
359,570,358
360,745,516
...,...,...
1188,1141,513
1198,566,840
1199,566,840
1200,566,840


In [17]:
# Скачать полную таблицу с рекомендациями можно расскоментировав код ниже и запустив ячейку.
# recommend_df.to_excel('recommendations.xlsx')

=======

### Выводы

1. Таблица рекомендаций составлена с учетом достаточной частоты сложившихся пар курсов (75% наименьших значений было отсеяно; ограничение частоты совместных покупок = 8 раз и более).<br>
2. Отсутствующие значения заменены предложенными двумя наиболее популярными курсами за последние полгода рассматриваемого периода (id 566 и 840).<br>
3. При необходимости, предложенные курсы можно легко заменить на другие, изменив значение переменных.<br>
4. Сценарий "два наиболее популярных за полгода" оправдал себя: самый популярный за весь период курс (id 551) оказался неактуальным. В коде реализована его замена для рекомендаций.<br>